In [1]:
from importlib.metadata import version
print("torch version:", version("torch"))

torch version: 2.6.0+cu124


# Ejercicio **evaluable**: Prepara tu dataset favorito

Si quieres probar con algo similar a `the-verdict.txt` pero en español, puedes usar los disponibles en este dataset de HuggingFace: https://huggingface.co/datasets/Fernandoefg/cuentos_es (aquí más info: https://www.linkedin.com/pulse/dataset-de-cuentos-en-espa%C3%B1ol-fernando-fuentes-gallegos-ssuyc/).

In [2]:
!pip install datasets

In [3]:
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 1

In [4]:
from datasets import load_dataset

# Cargamos el dataset de cuentos en español
cuentos = load_dataset("Fernandoefg/cuentos_es", split="train")

# Mostramos los títulos disponibles
for i in range(5):
    print(f"Cuento {i+1}: {cuentos[i]['title']}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/221 [00:00<?, ?B/s]

dataset_cuentos_es.zip:   0%|          | 0.00/56.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7239 [00:00<?, ? examples/s]

Cuento 1: Basilisa la Hermosa
Cuento 2: Dios y el párroco
Cuento 3: El adivino
Cuento 4: El campesino, el oso y la zorra
Cuento 5: El corredor veloz


### Seleccionamos el cuento

In [5]:
cuento = cuentos[2]['content']
cuento[:500] # Muestra los primeros 500 caracteres

'Era un campesino pobre y muy astuto apodado Escarabajo, que quería adquirir fama de adivino.\nUn día robó una sábana a una mujer, la escondió en un montón de paja y se empezó a alabar diciendo que estaba en su poder el adivinarlo todo. La mujer lo oyó y vino a él pidiéndole que adivinase dónde estaba su sábana. El campesino le preguntó:\n-¿Y qué me darás por mi trabajo?\n-Un pud de harina y una libra de manteca.\n-Está bien.\nSe puso a hacer como que meditaba, y luego le indicó el sitio donde estaba '

### Ahora tokenizamos el texto


In [6]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', cuento)
preprocessed = [item for item in preprocessed if item and not item.isspace()]
print(preprocessed[:40])
print("Número de tokens:", len(preprocessed))

['Era', 'un', 'campesino', 'pobre', 'y', 'muy', 'astuto', 'apodado', 'Escarabajo', ',', 'que', 'quería', 'adquirir', 'fama', 'de', 'adivino', '.', 'Un', 'día', 'robó', 'una', 'sábana', 'a', 'una', 'mujer', ',', 'la', 'escondió', 'en', 'un', 'montón', 'de', 'paja', 'y', 'se', 'empezó', 'a', 'alabar', 'diciendo', 'que']
Número de tokens: 1040


### Crear vocabulario y diccionario

In [7]:
all_words = sorted(set(preprocessed))
vocab = {token: idx for idx, token in enumerate(all_words)}
print(f"Tamaño del vocabulario: {len(vocab)}")
# Mostramos los primeros 20 elementos
for i, (token, idx) in enumerate(vocab.items()):
    print(f"{token}: {idx}")
    if i >= 19:
        break

Tamaño del vocabulario: 400
!: 0
,: 1
-Bueno: 2
-Envía: 3
-Escarabajo: 4
-Espera: 5
-Está: 6
-Nuestras: 7
-Oye: 8
-Que: 9
-Si: 10
-Sí: 11
-Un: 12
-contestó: 13
-dijo: 14
-exclamó: 15
-¡Es: 16
-¡Gracias: 17
-¡Hola: 18
-¡Oh: 19


### Cogemos la clase del tokenizador

In [8]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item and not item.isspace()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?\!"()\'])', r'\1', text)
        return text

tokenizer = SimpleTokenizerV1(vocab)

# Probar con una frase del cuento
texto_prueba = cuento[:120]
ids = tokenizer.encode(texto_prueba)
print("IDs:", ids)
print("Decodificado:", tokenizer.decode(ids))

IDs: [40, 372, 94, 295, 386, 257, 82, 79, 41, 1, 314, 315, 69, 187, 126, 68, 23, 55, 152, 325, 373, 351, 58, 372]
Decodificado: Era un campesino pobre y muy astuto apodado Escarabajo, que quería adquirir fama de adivino. Un día robó una sábana a un


<br>
<br>
<br>
<br>



# Ejercicio **evaluable**: Generar texto

1. Utilizad el método `tokenizer.encode` para preparar algún texto de entrada (a vuestra elección)
2. A continuación, convertid este texto en un tensor pytpoch a través de (`torch.tensor(...)`)
3. Añadid la dimensión de batch mediante `.unsqueeze(0)`.
4. Utilizad la función `generate_text_simple` para que el modelo genere algún texto basado en el texto que le hayáis introducido
5. La salida del paso 4 serán los IDs de los tokens, que tendréis que convertir de nuevo en texto mediante el método `tokenizer.decode`

In [9]:
!pip install tiktoken

In [14]:
# 1. Codificamos el texto de entrada
input_text = "Escarabajo"
input_ids = tokenizer.encode(input_text)
print("Token IDs:", input_ids)



# 2. Convertimos a tensor de PyTorch
import torch
input_tensor = torch.tensor(input_ids, dtype=torch.long)
print("Tensor shape:", input_tensor.shape)



# 3. Añadimos la dimensión de batch
input_tensor = input_tensor.unsqueeze(0)
print("Tensor con batch:", input_tensor.shape)



# 4. Función generate_text_simple (idéntica a la del notebook, pero corregida para softmax)
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]  # (batch, vocab_size)
        probas = torch.softmax(logits, dim=-1)
        next_id = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)
        idx = torch.cat([idx, next_id], dim=1)
    return idx



# 5. Definición del modelo GPT igual que en el notebook original
import torch.nn as nn

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop = nn.Dropout(cfg["emb_dropout"])
        self.ln = nn.LayerNorm(cfg["emb_dim"])
        self.head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.tok_emb(idx)  # (B, T, emb_dim)
        pos = torch.arange(T, device=idx.device)
        pos_emb = self.pos_emb(pos)[None, :, :]  # (1, T, emb_dim)
        x = tok_emb + pos_emb
        x = self.drop(x)
        x = self.ln(x)
        logits = self.head(x)  # (B, T, vocab_size)
        return logits



# 6. Ejemplo de configuración igual que en el notebook original
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "emb_dim": 768,
    "context_length": 1024,
    "emb_dropout": 0.1,
}
model = GPTModel(GPT_CONFIG_124M)



# 7. Generamos texto con el modelo
model.eval()
max_new_tokens = 20
context_size = model.pos_emb.num_embeddings

with torch.no_grad():
    generated_ids = generate_text_simple(model, input_tensor, max_new_tokens, context_size)

print("IDs generados:", generated_ids)



# 8. Decodificamos los IDs generados a texto
generated_text = tokenizer.decode(generated_ids[0].tolist())
print("Texto generado:")
print(generated_text)


Token IDs: [41]
Tensor shape: torch.Size([1])
Tensor con batch: torch.Size([1, 1])
IDs generados: tensor([[   41, 28212, 44078, 34089,  4558, 30001, 30840, 10167, 41428, 23630,
         39307, 27010, 17705,  1663, 11708,   115,  7103, 23638,  5696,  4366,
         47423]])


KeyError: 28212

In [24]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [25]:
import torch.nn as nn


class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)


class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # Shape: (b, num_tokens, d_out)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):

        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x



class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [32]:
import torch
import tiktoken
input_text = "Messi"
tokenizer = tiktoken.get_encoding("gpt2")

batch = []
batch.append(torch.tensor(tokenizer.encode(input_text)))
batch = torch.stack(batch, dim=0)

In [33]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[36479,    72]])

Output shape: torch.Size([1, 2, 50257])
tensor([[[ 0.4799, -0.5517, -0.8005,  ..., -0.2912,  0.1599, -0.5340],
         [ 0.2789,  0.0522,  0.0361,  ..., -0.6884,  0.1483,  0.6403]]],
       grad_fn=<UnsafeViewBackward0>)


In [34]:

# Ejercicio evaluable: Generar texto con el modelo implementado

# 1. Codificamos el texto de entrada

input_ids = tokenizer.encode(input_text)
print("Token IDs:", input_ids)

# 2. Convertimos a tensor de PyTorch
import torch
input_tensor = torch.tensor(input_ids, dtype=torch.long)
print("Tensor shape:", input_tensor.shape)

# 3. Añadimos la dimensión de batch
input_tensor = input_tensor.unsqueeze(0)
print("Tensor con batch:", input_tensor.shape)

# 4. Usamos la función generate_text_simple para generar texto
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]  # (batch, vocab_size)
        probas = torch.softmax(logits, dim=-1)
        next_id = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)
        idx = torch.cat([idx, next_id], dim=1)
    return idx

# 5. Generamos texto con el modelo
model.eval()
max_new_tokens = 20
context_size = model.pos_emb.num_embeddings

with torch.no_grad():
    generated_ids = generate_text_simple(model, input_tensor, max_new_tokens, context_size)

print("IDs generados:", generated_ids)

# 6. Decodificamos los IDs generados a texto
generated_text = tokenizer.decode(generated_ids[0].tolist())
print("Texto generado:")
print(generated_text)


Token IDs: [36479, 72]
Tensor shape: torch.Size([2])
Tensor con batch: torch.Size([1, 2])
IDs generados: tensor([[36479,    72, 15489, 22364, 37754, 23127, 10936,  7774, 30704, 17054,
         36647, 10266, 44680, 32870,  1033,  6229, 36045,  4156, 14921, 20766,
         24978, 39782]])
Texto generado:
Messi bay stereotypes VMwareDM Garyappy 1935 tribal Ministers judgesPOR fetish exp ment facilitating apparent arranged ampunciation scriptures


### Ejercicio 3 y 4

In [37]:
# 1. Codificamos el texto de entrada
input_text = "Escarabajo"
input_ids = tokenizer.encode(input_text)
print("Token IDs:", input_ids)



# 2. Convertimos a tensor de PyTorch
import torch
input_tensor = torch.tensor(input_ids, dtype=torch.long)
print("Tensor shape:", input_tensor.shape)



# 3. Añadimos la dimensión de batch
input_tensor = input_tensor.unsqueeze(0)
print("Tensor con batch:", input_tensor.shape)



# 4. Función generate_text_simple (idéntica a la del notebook, pero corregida para softmax)
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]  # (batch, vocab_size)
        probas = torch.softmax(logits, dim=-1)
        next_id = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)
        idx = torch.cat([idx, next_id], dim=1)
    return idx



# 5. Definición del modelo GPT igual que en el notebook original
import torch.nn as nn

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop = nn.Dropout(cfg["emb_dropout"])
        self.ln = nn.LayerNorm(cfg["emb_dim"])
        self.head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.tok_emb(idx)  # (B, T, emb_dim)
        pos = torch.arange(T, device=idx.device)
        pos_emb = self.pos_emb(pos)[None, :, :]  # (1, T, emb_dim)
        x = tok_emb + pos_emb
        x = self.drop(x)
        x = self.ln(x)
        logits = self.head(x)  # (B, T, vocab_size)
        return logits



# 6. Ejemplo de configuración igual que en el notebook original
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "emb_dim": 768,
    "context_length": 1024,
    "emb_dropout": 0.1,
}

# 7. Cargamos el modelo desde el archivo model.pth
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(torch.load("model.pth", map_location="cpu"))
model.eval()



# 8. Generamos texto con el modelo
max_new_tokens = 20
context_size = model.pos_emb.num_embeddings

with torch.no_grad():
    generated_ids = generate_text_simple(model, input_tensor, max_new_tokens, context_size)

print("IDs generados:", generated_ids)



# 9. Decodificamos los IDs generados a texto
def safe_decode(tokenizer, ids):
    return " ".join([tokenizer.int_to_str.get(i, f"<UNK:{i}>") for i in ids])

generated_text = safe_decode(tokenizer, generated_ids[0].tolist())
print("Texto generado:")
print(generated_text)


Token IDs: [123]
Tensor shape: torch.Size([1])
Tensor con batch: torch.Size([1, 1])


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

ME DA ESTE ERROR PORQUE TARDA TANTO EN ENTRENAR EL MODELO QUE LO HE CORTADO (LLEVABA YA COMO 4 HORAS Y PICO), POR LO QUE EL ARCHIVO MODEL.PTH ESTARÁ ERRONEO

### Ejercicio 5

In [38]:
# 1. Texto de entrenamiento sobre fútbol
train_text = """
El fútbol es el deporte más popular del mundo. Se juega en equipos de once jugadores y el objetivo es marcar goles.
Equipos como el Real Madrid, el Barcelona, el Manchester United y el Bayern Múnich han hecho historia.
Los mundiales son eventos donde selecciones nacionales compiten por el título más importante.
"""

# 2. Tokenizador simple (puedes usar el tuyo si prefieres)
class SimpleTokenizer:
    def __init__(self):
        words = sorted(set(train_text.lower().split()))
        self.str_to_int = {word: i+1 for i, word in enumerate(words)}
        self.str_to_int["<PAD>"] = 0
        self.int_to_str = {i: w for w, i in self.str_to_int.items()}

    def encode(self, text):
        return [self.str_to_int.get(w, 0) for w in text.lower().split()]

    def decode(self, ids):
        return " ".join([self.int_to_str.get(i, "<UNK>") for i in ids])

tokenizer = SimpleTokenizer()
data = tokenizer.encode(train_text)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 3. Crear dataset
class FootballDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx+self.block_size])
        y = torch.tensor(self.data[idx+1:idx+self.block_size+1])
        return x, y

block_size = 8
dataset = FootballDataset(data, block_size)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

# 4. Modelo GPT (igual que antes)
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop = nn.Dropout(cfg["emb_dropout"])
        self.ln = nn.LayerNorm(cfg["emb_dim"])
        self.head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.tok_emb(idx)
        pos = torch.arange(T, device=idx.device)
        pos_emb = self.pos_emb(pos)[None, :, :]
        x = tok_emb + pos_emb
        x = self.drop(x)
        x = self.ln(x)
        logits = self.head(x)
        return logits

# 5. Configuración
vocab_size = len(tokenizer.str_to_int)
GPT_CONFIG = {
    "vocab_size": vocab_size,
    "emb_dim": 64,
    "context_length": block_size,
    "emb_dropout": 0.1,
}

model = GPTModel(GPT_CONFIG)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# 6. Entrenamiento simple
model.train()
for epoch in range(50):
    total_loss = 0
    for x, y in loader:
        logits = model(x)
        loss = loss_fn(logits.view(-1, vocab_size), y.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss:.4f}")

# 7. Guardar el modelo entrenado (opcional)
torch.save(model.state_dict(), "football_model.pth")

# 8. Generar texto
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        probas = torch.softmax(logits, dim=-1)
        next_id = torch.argmax(probas, dim=-1, keepdim=True)
        idx = torch.cat([idx, next_id], dim=1)
    return idx

model.eval()
start_text = "el fútbol"
input_ids = tokenizer.encode(start_text)
input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0)

generated_ids = generate_text_simple(model, input_tensor, max_new_tokens=20, context_size=block_size)
generated_text = tokenizer.decode(generated_ids[0].tolist())

print("Texto generado:")
print(generated_text)


Epoch 0, Loss: 43.1538
Epoch 10, Loss: 22.0322
Epoch 20, Loss: 9.1307
Epoch 30, Loss: 5.2633
Epoch 40, Loss: 4.3486
Texto generado:
el fútbol es marcar goles. equipos como el real madrid, el real madrid, el real madrid, el real madrid, el real madrid,


### Ejercicio 6

In [40]:
from transformers import pipeline

# Cargar el modelo de summarization
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

#Texto a resumir
texto = """
NLP (Natural Language Processing) es una rama de la inteligencia artificial (IA) que permite a las computadoras comprender, interpretar y generar lenguaje humano. En esencia, se trata de un conjunto de técnicas y algoritmos que permiten a las máquinas interactuar con los humanos de manera natural, utilizando lenguaje hablado o escrito.
El NLP permite:
Comprender el lenguaje humano:
Las máquinas pueden analizar textos y conversaciones para extraer información, identificar el significado y la intención del hablante, y comprender el contexto.
Generar lenguaje humano:
Las máquinas pueden producir texto o voz de forma natural, utilizando la gramática y el vocabulario correctos, y adaptando el lenguaje a la audiencia y el contexto.
Traducir lenguajes:
El NLP permite a las máquinas traducir texto entre diferentes idiomas, utilizando algoritmos para identificar las palabras y frases correspondientes en cada lengua.
Ejemplos de aplicaciones del NLP:
Asistentes virtuales:
Siri, Alexa, Google Assistant, entre otros, utilizan el NLP para entender las consultas de los usuarios y responder en lenguaje natural.
Búsqueda en Internet:
Google y otros motores de búsqueda utilizan el NLP para comprender las consultas de los usuarios y ofrecer resultados más relevantes.
Chatbots:
Los chatbots utilizan el NLP para interactuar con los usuarios y responder preguntas de manera automática.
Traducción automática:
Las aplicaciones de traducción automática, como Google Translate, utilizan el NLP para traducir texto entre diferentes idiomas.
Análisis de sentimientos:
El NLP se utiliza para analizar la opinión de los usuarios sobre productos, servicios o eventos, a través de texto o voz.
"""

#Ejecutar el resumen
resumen = summarizer(texto, max_length=130, min_length=50, do_sample=False)

#Mostrar resultado
print("Resumen generado:")
print(resumen[0]['summary_text'])


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


📝 Resumen generado:
NLP (Natural Language Processing) es una rama de la inteligencia artificial (IA) permite a las computadoras comprender, interpretar and generar lenguaje humano. En esencia, se trata of un conjunto of técnicas y algoritmos that permiten a las máquinas interactuar with los humanos.
